A1 de Linguagens de Programação – 28/09/2017
--
Professor: Renato Rocha Souza  
--

Nome do Aluno(a):

In [ ]:
meu_nome = input('Entre com o seu nome completo: ')
print('Olá, {}!'.format(meu_nome))
print('Bom trabalho!')

In [1]:
# Somente estes módulos podem ser importados para a realização das questões
import numpy as np
import matplotlib.pyplot as plt
import requests
import random
import re
import string
import time
from bs4 import BeautifulSoup as bs
from IPython.display import clear_output
%matplotlib inline

Questão 1 (1,5 pontos)  


Mostre graficamente (usando matplotlib) que as funções:  
$ y = 2x^2 - 10x -32$  
e  
$ y = 2x^3 - 12x^2 +4x +2$  
possuem, respectivamente, 2 e 3 raízes reais e distintas.  

In [ ]:
x = np.linspace(-100, 100, 10000)
y1 = 2*x**2 - 10*x - 32
y2 = 2*x**3 - 12*x**2 + 4*x +2
y3 = np.zeros(10000)

fig, ax = plt.subplots(1,2, figsize=(12,8))
ax[0].plot(x,y1)
ax[0].plot(x,y3)
ax[0].set_xlim([-30,30])
ax[0].set_ylim([-55,400])

ax[1].plot(x,y2)
ax[1].plot(x,y3)
ax[1].set_xlim([-10,12])
ax[1].set_ylim([-50,10])

plt.show()

Questão 2 (2.5 pontos)  

Partindo do código abaixo, escreva comandos para realizar as seguintes tarefas:
- Identificar as 10 palavras mais frequentes, independente de possuírem letras maiúsculas ou minúsculas;
- Identificar os trechos em que há uma citação de fala de personagens (entre aspas);
- Identificar as 5 letras mais comuns no final de palavras;
- Salvar os resultados em um arquivo chamado "respostas_q2.txt" no computador

In [4]:
url = 'http://www.releituras.com/guimarosa_margem.asp'
page = requests.get(url)
html = page.text
sopa = bs(html, "lxml")
texto = sopa.text


In [ ]:
#primeiro item

texto = texto.replace('\n',' ')
texto = texto.replace('\r',' ')
texto = re.sub(' +',' ', texto)  #elimina múltiplos espaços em branco
lista = texto.split()
lista = [l.lower().strip(string.punctuation) for l in lista]

def freqdist(iterable):
    dic_freq = {}
    for elem in iterable:
        if elem in dic_freq:
            dic_freq[elem] += 1
        else:
            dic_freq[elem] = 1
    return dic_freq

freqs = freqdist(lista)

#https://stackoverflow.com/questions/613183/how-to-sort-a-dictionary-by-value
ordered_freqs = sorted(freqs.items(), key=lambda x: x[1]) 
#print(ordered_freqs)

#segundo item

#https://stackoverflow.com/questions/2402797/regex-find-characters-between
citacoes = re.findall(r'"(.*?)"',texto)
#print(citacoes)

#terceiro item

ultimas = [l for l in lista if len(l) > 0]
ultimas = [l[-1] for l in ultimas]
freqs2 = freqdist(ultimas)
ordered_freqs2 = sorted(freqs2.items(), key=lambda x: x[1])
#print(ordered_freqs2)

#quarto item

with open('respostas_q2.txt','w') as f:
    for key,value in ordered_freqs:
        f.write('{},{}\n'.format(key,value))
    f.write('\n')
    for citacao in citacoes:
        f.write('{}\n'.format(citacao))
    f.write('\n')
    for key,value in ordered_freqs2:
        f.write('{},{}\n'.format(key,value))

Questão 3 (3 pontos)  

Christian Goldbach propos que qualquer número ímpar não primo pode ser formado pela soma de um número primo e duas vezes o quadrado de outro número.

$9 = 7 + 2×1^2$  
$15 = 7 + 2×2^2$  
$21 = 3 + 2×3^2$  
$25 = 7 + 2×3^2$  
$27 = 19 + 2×2^2$  
$33 = 31 + 2×1^2$  

A conjectura se mostrou falsa. 

Crie um programa para mostrar qual o primeiro número ímpar não primo que não pode ser escrito pela soma de um primo e duas vezes o quadrado de outro número.

In [ ]:
%%time

max_impar = 10000
max_quad = 100

def is_prime(n):
    if n <= 3:
        return n >= 2
    if n % 2 == 0 or n % 3 == 0:
        return False
    for i in range(5, int(n ** 0.5) + 1, 6):
        if n % i == 0 or n % (i + 2) == 0:
            return False
    return True


impares = set(range(1,max_impar,2))
primos = set(filter(is_prime,impares))
impares_nao_primos = impares - primos
impares_nao_primos.discard(1)
fatores = [2 * x**2 for x in range(1,max_quad+1)]

nlins = len(primos)
ncols = len(fatores)
array = np.zeros((nlins,ncols))
for i in range(nlins):
    for j in range(ncols):
        array[i,j] =  list(primos)[i] + fatores[j]

#https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.unique.html
#https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.flatten.html
somas = np.unique(array.flatten())

#https://stackoverflow.com/questions/7088625/what-is-the-most-efficient-way-to-check-if-a-value-exists-in-a-numpy-array
for numero in impares_nao_primos:
    if not np.any(somas == numero):
        print(numero)
        break

In [8]:
def is_prime(n):
    if n <= 3:
        return n >= 2
    if n % 2 == 0 or n % 3 == 0:
        return False
    for i in range(5, int(n ** 0.5) + 1, 6):  ## Começando do 5 de 6 em 6 (porque 6 é 2*3)... vai achando todo os números
        if n % i == 0 or n % (i + 2) == 0:
            return False
    return True

is_prime(0)

False

Questão 4 (3 pontos) 

Vamos ajudar uma tartaruga a sair de um labirinto?  

Implemente em Python uma classe chamada “Tartaruga” com as seguintes propriedades:  

- Recebe um labirinto (array do numpy) com 0s nas células desocupadas e 1s nas células ocupadas;
- Sorteia, ao iniciar, uma posição aleatória para si neste array (representada por 2) dentre as células com 0 (não ocupadas);  
- Possui um método "andar", que recebe as ordens:
    - Norte, na qual tenta diminuir o índice da linha, mantendo o da coluna;
    - Sul, na qual tenta aumentar o índice da linha, mantendo o da coluna;
    - Leste, na qual tenta aumentar o índice da coluna, mantendo o da linha;
    - Oeste, na qual tenta diminuir o índice da coluna, mantendo o da linha;
    - Só pode andar para posições que estão ocupadas com '0'.
- Possui um método para mostrar a sua posição no labirinto;
- Deve guardar as posições que já visitou e a quantidade de vezes (com método para mostrar este array);
- Possui um método "tenta_sair" com uma estratégia:  
    - Anda para o norte, enquanto for possível; 
    - Quando não for possível andar para o norte, anda para o sul;
    - Quando não for possível andar para o norte ou o sul, anda para o leste;
    - Quando não for possível andar para o norte ou o sul ou o leste, anda para oeste;
    - Quando passar por uma célula já visitada, deve mudar a sequência de "tenta_sair" na forma:  
        NSLO -> SLON -> LONS -> ONSL -> ...  
- Deve saber a cada passo se saiu do labirinto, mostrando uma mensagem e interrompendo o processo de tentar sair;
- Caso não consiga sair em 1000 iterações, deve declarar que é impossível sair.  

In [ ]:
#exemplo de labirinto

labirinto = np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
                      [1,0,0,0,1,0,0,0,1,1,0,1,1,0,0,0,0,0,1,0,0,0],
                      [1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,1,1],
                      [1,0,1,0,1,0,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1],
                      [1,0,0,0,1,0,0,1,1,0,0,1,1,1,1,0,0,0,1,0,1,1],
                      [1,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,1],
                      [1,0,1,0,1,0,0,1,0,0,0,1,0,1,1,1,0,0,1,0,1,1],
                      [1,0,0,0,1,0,0,1,1,0,0,0,0,1,1,0,1,0,1,0,1,1],
                      [1,0,1,0,1,0,0,1,1,0,0,1,1,1,1,0,0,0,1,0,1,1],
                      [1,1,1,0,1,1,1,1,1,1,0,0,0,0,1,1,1,0,1,0,0,1],
                      [1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1],
                      [1,0,1,1,1,0,1,1,1,1,1,1,0,0,0,1,1,1,1,1,0,1],
                      [1,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,1,1,1,1,0,1],
                      [1,0,1,0,1,1,1,1,0,1,0,1,1,1,0,0,0,0,1,1,0,1],
                      [1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,1,0,1,1,0,1],
                      [1,0,1,1,0,1,1,1,1,1,0,1,0,0,0,1,1,0,1,1,0,1],
                      [1,0,1,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,1,1,0,1],
                      [1,1,1,1,1,0,1,1,1,0,1,1,0,0,0,1,1,1,1,1,0,1],
                      [1,0,0,0,0,0,1,0,0,0,1,1,1,1,0,1,1,0,0,1,0,1],
                      [1,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1],
                      [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1],
                      [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1]], dtype=int)

labirinto.shape

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))
ax.matshow(labirinto, cmap=plt.cm.Blues)

In [ ]:
class Tartaruga:
    def __init__(self,labirinto):
        self.labirinto = labirinto.copy()
        self.dim = labirinto.shape
        self.posicoes = labirinto.copy()
        while True:
            self.linha = np.random.randint(labirinto.shape[0])
            self.coluna = np.random.randint(labirinto.shape[1])
            if labirinto[self.linha,self.coluna] == 0:
                self.labirinto[self.linha,self.coluna] = 2
                break
                
    def mostra_posicao(self):
        fig, ax = plt.subplots(figsize = (8,8))
        ax.matshow(self.labirinto, cmap=plt.cm.Blues)
    
    def mostra_visitas(self):
        fig, ax = plt.subplots(figsize = (8,8))
        ax.matshow(self.posicoes, cmap=plt.cm.inferno)
        
    def guarda_visitas(self):
        self.posicoes[self.linha,self.coluna] -= 1
                
    def saiu(self, linha, coluna):
        if linha == 0 or coluna == 0 or linha == self.labirinto.shape[0]-1 or coluna == self.labirinto.shape[1]-1:
            return True
        else:
            return False
        
    def parede(self, linha, coluna):
        if self.labirinto[linha,coluna] == 1:
            return True
                
    def andar(self, direcao):
        if direcao == 'N' and not self.parede(self.linha-1,self.coluna):
            self.labirinto[self.linha,self.coluna] = 0
            self.linha -= 1
            self.labirinto[self.linha,self.coluna] = 2
            self.guarda_visitas()
        elif direcao == 'S' and not self.parede(self.linha+1,self.coluna):
            self.labirinto[self.linha,self.coluna] = 0
            self.linha += 1
            self.labirinto[self.linha,self.coluna] = 2
            self.guarda_visitas()
        elif direcao == 'L' and not self.parede(self.linha,self.coluna+1):
            self.labirinto[self.linha,self.coluna] = 0
            self.coluna += 1
            self.labirinto[self.linha,self.coluna] = 2
            self.guarda_visitas()
        elif direcao == 'O' and not self.parede(self.linha,self.coluna-1):
            self.labirinto[self.linha,self.coluna] = 0
            self.coluna -= 1
            self.labirinto[self.linha,self.coluna] = 2
            self.guarda_visitas()

    def tenta_sair(self,iteracoes):
        for i in range(iteracoes):
            direcao = random.choice(list('NSLO'))
            self.andar(direcao)
            if self.saiu(self.linha, self.coluna):
                print('Tartaruga Saiu!!')
                break
                self.mostra_posicao()
        if not self.saiu(self.linha, self.coluna):
            print('Tartaruga não conseguiu sair')
    
# O algoritmo originalmente pedido não garantia que a tartaruga saísse...
#
#    def andar(self, direcao):
#        if direcao == 'N' and not self.parede(self.linha-1,self.coluna):
#            self.labirinto[self.linha,self.coluna] = 0
#            self.linha -= 1
#            self.labirinto[self.linha,self.coluna] = 2
#            self.guarda_visitas()
#            return True
#        elif direcao == 'S' and not self.parede(self.linha+1,self.coluna):
#            self.labirinto[self.linha,self.coluna] = 0
#            self.linha += 1
#            self.labirinto[self.linha,self.coluna] = 2
#            self.guarda_visitas()
#            return True
#        elif direcao == 'L' and not self.parede(self.linha,self.coluna+1):
#            self.labirinto[self.linha,self.coluna] = 0
#            self.coluna += 1
#            self.labirinto[self.linha,self.coluna] = 2
#            self.guarda_visitas()
#            return True
#        elif direcao == 'O' and not self.parede(self.linha,self.coluna-1):
#            self.labirinto[self.linha,self.coluna] = 0
#            self.coluna -= 1
#            self.labirinto[self.linha,self.coluna] = 2
#            self.guarda_visitas()
#            return True
#        return False
#
#
#        def tenta_sair(self):
#            for i in range(100):
#                self.mostra_posicao()
#                sucesso = True
#                while sucesso:
#                    sucesso = self.andar('N')
#                sucesso = True    
#                while sucesso:
#                    sucesso = self.andar('L')
#                sucesso = True    
#                while sucesso:
#                    sucesso = self.andar('S')
#                sucesso = True    
#                while sucesso:
#                    sucesso = self.andar('O')
#                #time.sleep(1)
#                clear_output()

In [ ]:
turtle = Tartaruga(labirinto)
turtle.mostra_posicao()
turtle.tenta_sair(10000)
turtle.mostra_posicao()
turtle.mostra_visitas()

In [ ]:
class Tartaruga:
    def __init__(self,labirinto):
        self.labirinto = np.copy(labirinto)  #guarda o labirinto na memória
        ja_visitada = []
        size = labirinto.size
        shape = labirinto.shape
        self.pos = (0,0)
        while len(ja_visitada)<size and self.labirinto[self.pos] == 1:
            self.pos = (r.randint(0,shape[0]-1),r.randint(0,shape[1]-1))
            while self.pos in ja_visitada:
                self.pos = (r.randint(0,shape[0]-1),r.randint(0,shape[1]-1))
        if self.labirinto[self.pos]==1:
            raise LabirintoTodoUm
        self.labirinto[self.pos] = 2
        self.ja_visitado = []
    
    def posicao(self):
        return self.pos
    
    def andar(self,direcao):
        prev=self.pos
        if direcao=='N':
            if self.pos[0] == 0:
                raise SaiuDoLabirinto
            tentativa = (self.pos[0]-1,self.pos[1])
            if self.labirinto[tentativa] == 0:
                self.pos = tentativa
                self.labirinto[prev]=0
                self.labirinto[self.pos]=2
                return True
            else:
                return False
        elif direcao=='S':
            if self.pos[0] == self.labirinto.shape[0]-1:
                raise SaiuDoLabirinto
            tentativa = (self.pos[0]+1,self.pos[1])
            if self.labirinto[tentativa] == 0:
                self.pos = tentativa
                self.labirinto[prev]=0
                self.labirinto[self.pos]=2
                return True
            else:
                return False
        elif direcao=='L':
            if self.pos[0] == self.labirinto.shape[1]-1:
                raise SaiuDoLabirinto
            tentativa = (self.pos[0],self.pos[1]+1)
            if self.labirinto[tentativa] == 0:
                self.pos = tentativa
                self.labirinto[prev]=0
                self.labirinto[self.pos]=2
                return True
            else:
                return False
        elif direcao=='O':
            if self.pos[0] == 0:
                raise SaiuDoLabirinto
            tentativa = (self.pos[0],self.pos[1]-1)
            if self.labirinto[tentativa] == 0:
                self.pos = tentativa
                self.labirinto[prev]=0
                self.labirinto[self.pos]=2
                return True
            else:
                return False

Questão Bônus (0,5 pontos)  

O problema de Monty Hall, também conhecido por paradoxo de Monty Hall é um problema matemático e paradoxo que surgiu a partir de um concurso televisivo dos Estados Unidos chamado "Let’s Make a Deal", exibido na década de 1970. O jogo consiste no seguinte: Monty Hall (o apresentador) apresentava 3 portas aos concorrentes, sabendo que atrás de uma delas está um carro (prêmio bom) e que as outras têm prêmios de pouco valor.

- Na 1ª etapa o concorrente escolhe uma porta (que ainda não é aberta);  
- Em seguida, Monty abre uma das outras duas portas que o concorrente não escolheu, sabendo de antemão que o carro não se encontra na porta que abrirá;  
- Agora com duas portas apenas para escolher — pois uma delas já se viu, na 2ª etapa, que não tinha o prêmio — e sabendo que o carro está atrás de uma delas, o concorrente tem que se decidir se permanece com a porta que escolheu no início do jogo e abre-a ou se muda para a outra porta que ainda está fechada para então a abrir.  

Demonstre graficamente que é mais provável ganhar o prêmio se trocar de porta do que se não o fizer. Quais as probabilidade de acertar, em ambos os casos?

In [ ]:
#seu código aqui